Name: Zihan Qin

Github: zihan-qin

USCID: 8130 9206 33


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error,accuracy_score
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings("ignore")
import os
import random
from sklearn.utils import resample
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

1.(a)-(c)ii

In [2]:
c_name = []
for j in range(1,7):
    for i in ['min','max','mean','median','std','first_quartile','third_quartile']:
        c_name.append(i+"_"+str(j))

In [3]:
class dataset:
    def __init__(self):
        self.dataset = pd.DataFrame(columns=c_name)
    def feature_extraction(self,path):
        tag = path.split('/')[-2]
        data = pd.read_csv(path,skiprows=4,error_bad_lines=False)
        data = data.drop(['# Columns: time'],axis=1)
        agg = data.aggregate(['min','max','mean','median','std'])
        q1 = {}
        q3 = {}
        for i in agg:
            q1[i] = np.percentile(data[i],25)
            q3[i] = np.percentile(data[i],75)
        agg = agg.append(q1,ignore_index=True)
        agg = agg.append(q3,ignore_index=True)
        agg.index = ['min','max','mean','median','std','first_quartile','third_quartile']
        tmp = pd.DataFrame(np.array(agg.T).ravel().reshape(1,-1),columns=c_name)
        tmp['y'] = tag
        self.dataset = self.dataset.append(tmp)

In [4]:
trainset = dataset()
testset = dataset()

In [5]:
path = '../data/AReM'
folders = next(os.walk(path))[1]
for i in folders:
    dirpath = path+'/'+i
    datacsv = datacsv = os.listdir(dirpath)
    if i == 'bending1' or i == 'bending2':
        for csv in datacsv:
            if csv == 'dataset1.csv' or csv == 'dataset2.csv':
                testset.feature_extraction(dirpath+'/'+csv)
            else:
                trainset.feature_extraction(dirpath+'/'+csv)
    else:
        for csv in datacsv:
            if csv == 'dataset1.csv' or csv == 'dataset2.csv' or csv == 'dataset3.csv':
                testset.feature_extraction(dirpath+'/'+csv)
            else:
                trainset.feature_extraction(dirpath+'/'+csv)
                

In [6]:
trainset = trainset.dataset
testset = testset.dataset

In [7]:
trainset.index = range(len(trainset))
testset.index = range(len(testset))
trainset = trainset.dropna()
testset = testset.dropna()

In [8]:
trainset

,min_1,max_1,mean_1,median_1,std_1,first_quartile_1,third_quartile_1,min_2,max_2,mean_2,...,first_quartile_5,third_quartile_5,min_6,max_6,mean_6,median_6,std_6,first_quartile_6,third_quartile_6,y
0,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,0.413125,...,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.8300,bending1
1,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,0.378083,...,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.8700,bending1
2,33.00,47.75,42.179812,43.50,3.670666,39.1500,45.00,0.0,3.00,0.696042,...,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.0000,bending1
3,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,0.535979,...,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.5000,bending1
4,35.00,47.40,43.954500,44.33,1.558835,43.0000,45.00,0.0,1.70,0.426250,...,35.3625,36.5000,0.0,1.79,0.493292,0.430,0.513506,0.0000,0.9400,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,18.50,44.25,35.752354,36.00,4.614802,33.0000,39.33,0.0,12.60,3.328104,...,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.0600,cycling
65,24.25,45.00,37.177042,36.25,3.581301,34.5000,40.25,0.0,8.58,2.374208,...,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.9000,cycling
66,23.33,43.50,36.244083,36.75,3.822016,33.4575,39.25,0.0,9.71,2.736021,...,15.7500,21.0000,0.0,11.15,3.530500,3.110,1.963685,2.1700,4.6175,cycling
67,26.25,44.25,36.957458,36.29,3.434863,34.5000,40.25,0.0,8.64,2.420083,...,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.0300,cycling


In [9]:
testset

,min_1,max_1,mean_1,median_1,std_1,first_quartile_1,third_quartile_1,min_2,max_2,mean_2,...,first_quartile_5,third_quartile_5,min_6,max_6,mean_6,median_6,std_6,first_quartile_6,third_quartile_6,y
0,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,0.358604,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.300,bending1
1,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,0.372437,...,32.0000,34.5000,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.300,bending1
2,19.33,43.50,34.227771,35.500,4.889576,30.5000,37.7500,0.0,14.50,3.995729,...,14.7500,18.6700,0.0,9.74,3.394125,3.100,1.792090,2.1050,4.425,walking
3,12.50,45.00,33.509729,34.125,4.850923,30.5000,36.7500,0.0,13.05,4.450771,...,14.6275,18.7500,0.0,8.96,3.378479,3.085,1.787360,2.0600,4.440,walking
4,15.00,46.75,34.660583,35.000,5.315110,31.0000,38.2500,0.0,13.44,4.200896,...,14.2500,18.5000,0.0,8.99,3.244396,3.000,1.630983,2.1200,4.240,walking
5,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,0.590833,...,20.5000,27.0000,0.0,4.97,0.700188,0.500,0.693720,0.4300,0.870,bending2
6,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,0.449708,...,15.0000,20.7500,0.0,6.76,1.122125,0.830,1.012342,0.4700,1.300,bending2
7,33.33,48.00,44.334729,45.000,2.476940,42.2500,46.5000,0.0,3.90,0.432958,...,9.3300,17.7500,0.0,5.02,0.933000,0.830,0.673609,0.4700,1.250,standing
8,35.50,46.25,43.174938,43.670,1.989052,42.5000,44.5000,0.0,2.12,0.506583,...,12.7500,16.5000,0.0,5.72,0.911979,0.830,0.666161,0.4700,1.220,standing
9,32.75,47.00,42.760562,44.500,3.398919,41.3300,45.3725,0.0,3.34,0.486167,...,13.0000,18.5650,0.0,5.73,0.842271,0.710,0.722165,0.4300,1.090,standing


In [10]:
total_data = trainset.append(testset)
total_data.index = range(len(total_data))
total_data

,min_1,max_1,mean_1,median_1,std_1,first_quartile_1,third_quartile_1,min_2,max_2,mean_2,...,first_quartile_5,third_quartile_5,min_6,max_6,mean_6,median_6,std_6,first_quartile_6,third_quartile_6,y
0,36.25,48.00,43.969125,44.500,1.618364,43.31,44.6700,0.0,1.50,0.413125,...,20.5000,23.7500,0.0,2.96,0.555312,0.49,0.487826,0.0000,0.83,bending1
1,37.00,48.00,43.454958,43.250,1.386098,42.50,45.0000,0.0,1.58,0.378083,...,22.2500,24.0000,0.0,5.26,0.679646,0.50,0.622534,0.4300,0.87,bending1
2,33.00,47.75,42.179812,43.500,3.670666,39.15,45.0000,0.0,3.00,0.696042,...,30.4575,36.3300,0.0,2.18,0.613521,0.50,0.524317,0.0000,1.00,bending1
3,33.00,45.75,41.678063,41.750,2.243490,41.33,42.7500,0.0,2.83,0.535979,...,28.4575,31.2500,0.0,1.79,0.383292,0.43,0.389164,0.0000,0.50,bending1
4,35.00,47.40,43.954500,44.330,1.558835,43.00,45.0000,0.0,1.70,0.426250,...,35.3625,36.5000,0.0,1.79,0.493292,0.43,0.513506,0.0000,0.94,bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,24.75,48.33,44.182937,48.000,7.495615,48.00,48.0000,0.0,3.11,0.101875,...,2.0000,5.5425,0.0,3.91,0.692771,0.50,0.675781,0.3225,0.94,lying
84,48.00,48.25,48.004167,48.000,0.032038,48.00,48.0000,0.0,0.43,0.007167,...,4.6700,10.0000,0.0,2.50,0.641229,0.50,0.388372,0.4600,0.83,lying
85,24.25,45.00,37.177042,36.250,3.581301,34.50,40.2500,0.0,8.58,2.374208,...,17.9500,21.7500,0.0,9.34,2.921729,2.50,1.852600,1.5000,3.90,cycling
86,28.75,44.75,37.561187,36.875,3.226507,35.25,40.2500,0.0,9.91,2.080687,...,18.0000,21.5000,0.0,9.62,2.765896,2.45,1.769203,1.4100,3.77,cycling


1(c)iii

In [11]:
data_features = total_data.drop('y',axis=1)

In [12]:
bs_std = []
for i in data_features:
    bs_res = bs.bootstrap(np.array(data_features[i]),stat_func=bs_stats.std,alpha=0.10)
    bs_std.append([i,bs_res.value,bs_res.lower_bound,bs_res.upper_bound])
    

In [13]:
bs_std = pd.DataFrame(bs_std,columns=['feature','std_estimation','lower_bound','upper_bound'])

In [14]:
bs_std

,feature,std_estimation,lower_bound,upper_bound
0,min_1,9.515445,8.332011,10.815473
1,max_1,4.369322,3.489754,5.417739
2,mean_1,5.305314,4.759182,5.916175
3,median_1,5.409056,4.863454,6.055942
4,std_1,1.762056,1.581636,1.961991
5,first_quartile_1,6.118526,5.621839,6.688131
6,third_quartile_1,5.109643,4.403458,5.907794
7,min_2,0.000000,0.000000,0.000000
8,max_2,5.033882,4.691976,5.467959
9,mean_2,1.565194,1.432257,1.744777


1(c)iv

In [15]:
bs_std.sort_values(by='std_estimation',ascending=False)

,feature,std_estimation,lower_bound,upper_bound
0,min_1,9.515445,8.332011,10.815473
5,first_quartile_1,6.118526,5.621839,6.688131
28,min_5,6.089107,4.724134,7.765815
33,first_quartile_5,6.061727,4.975031,7.357504
31,median_5,5.780655,4.715322,7.105454
29,max_5,5.708524,4.903095,6.682500
30,mean_5,5.643253,4.583011,6.884760
34,third_quartile_5,5.500200,4.483539,6.671262
3,median_1,5.409056,4.863454,6.055942
2,mean_1,5.305314,4.759182,5.916175


In [16]:
data_features.aggregate(func='std').sort_values(ascending=False)

min_1               9.569975
first_quartile_1    6.153590
min_5               6.124001
first_quartile_5    6.096465
median_5            5.813782
max_5               5.741238
mean_5              5.675593
third_quartile_5    5.531720
median_1            5.440054
mean_1              5.335718
third_quartile_1    5.138925
max_2               5.062729
max_3               4.875137
max_1               4.394362
first_quartile_3    4.220658
third_quartile_3    4.171628
median_3            4.036396
mean_3              4.008380
min_3               2.956462
max_6               2.518921
max_4               2.183625
third_quartile_2    2.125266
std_1               1.772153
mean_2              1.574164
third_quartile_4    1.552504
third_quartile_6    1.523599
median_2            1.412244
mean_4              1.166114
mean_6              1.154812
median_4            1.145586
median_6            1.086474
std_5               1.024898
std_3               0.946710
first_quartile_2    0.946386
std_2         

Since the variance can include the information of predicting the difference of dependent variable, According to the features which have the largest variance(min_1, first_quartile_1, min_5, first_quartile_5, median_5), I think is min, first quartile, median may be the three important features.

**ISLR 3.7.4**

I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. $Y = β_0 +β_1X +β_2X^2 +β_3X^3 +ε$.

(a) Suppose that the true relationship between X and Y is linear, i.e. $Y = β_0 + β_1X + ε$. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

I will expect the cubic regression model have lower RSS. Because the coefficients of $X^2$ and $X^3$ can be 0 in the best case. However, the $X^2$ and $X^3$ might fit some error from train data. In that case, the cubic regression model's training RSS will be lower than training RSS of linear regression model.

(b) Answer (a) using test rather than training RSS.

I will expect the test RSS of linear regression model lower than test RSS of cubic regression model since the cubic regression model will overfit the trainset.

(c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

I will expect the cubic regression model have lower RSS. If the real relationship between X and Y are more closed to quadratic or cubic, the cubic regression model will definitely fit better and have lower train RSS. Else, the real relationship between X and Y are more closed to linear, it will become the situation discussed in question(a), the cubic regression model still have lower train RSS.

(d) Answer (c) using test rather than training RSS.

There is not enough information to tell. If the real relationship between X and Y are closer to linear, the linear regression model will has the less test RSS, while the cubic regression model will has the less test RSS if the real relationship between X and Y are closer to cubic.